### Preliminaries

In [ ]:
import sys
import numpy as np
sys.path.append('../..')

import main
from utils import proputils as pu
from names import GlobNames as gn

%matplotlib widget

### Read macro input file

In [ ]:
props = pu.parse_file('macro.pro')


### Solve two-scale problem

In [ ]:
globdat = main.jive(props)

### Visualize macro-scale results

In [ ]:
viewmodule = globdat[gn.MODULEFACTORY].get_module('View','view')

viewprops = { 'view': {}}
viewprops['view']['plot'] = 'solution[dx]'
viewprops['view']['interactive'] = 'True'
viewprops['view']['constantLevels'] = 'False'
viewprops['view']['constantTicks'] = 'True'
viewprops['view']['step0'] = 4
viewprops['view']['ncolors'] = 10

viewmodule.init(viewprops, globdat)
viewmodule.plot(globdat)

### Visualize microscale results

In [ ]:
microglobdats = globdat[gn.MODEL]._models[0]._mat._globdats

In [ ]:
viewmodule.init(viewprops, microglobdats[0])
viewmodule.plot(microglobdats[0])